# 下载cifar-10数据
git clone https://github.com/tensorflow/models.git

cd models/tutorials/image/cifar10

In [19]:
import cifar10,cifar10_input
import tensorflow as tf
import numpy as np
import time

In [20]:
max_steps = 3000
batch_size = 128
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'

In [21]:
# 定义权重的初始化方法，w1为正则化参数，不为0则使用正则化
def variable_with_weight_loss(shape, stddev,w1):
    var = tf.Variable(tf.truncated_normal(shape,stddev = stddev))
    if w1 is not None:     
        # tf.nn.l2_loss()：output = sum(t ** 2) / 2
        # elsm-size multiply
        weight_loss = tf.multiply(tf.nn.l2_loss(var),w1,
                                  name = 'weight_loss')
        # 向默认图中添加collection，“零存整取”思想
        # http://blog.csdn.net/shenxiaolu1984/article/details/52815641
        tf.add_to_collection('losses',weight_loss)
    return var

# 下载cifar10数据

In [22]:
cifar10.maybe_download_and_extract()

产生训练需要的数据，包括特征及其对应的label,返回已封装好的tensor
这里对数据进行了Data Augmentation(数据增强)

In [23]:
images_train,labels_train = cifar10_input.distorted_inputs(
    data_dir = data_dir,batch_size = batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


# 归一化图片到24x24大小

In [24]:
images_test,labels_test = cifar10_input.inputs(eval_data = True,
                                               data_dir = data_dir,
                                               batch_size = batch_size)

# 输入x,y，24x24x3大小图片

In [25]:
image_holder = tf.placeholder(tf.float32,[batch_size,24,24,3])
label_holder = tf.placeholder(tf.int32,[batch_size])

# CONV1,64卷积核5x5

In [26]:
# 第一层 不使用正则化，故 w1 = 0
weight1 = variable_with_weight_loss(shape = [5,5,3,64],stddev = 5e-2,
                                   w1 = 0.0)
kernel1 = tf.nn.conv2d(image_holder,weight1,[1,1,1,1],padding = 'SAME')
bias1 = tf.Variable(tf.constant(0.0,shape = [64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1,bias1))
# 池化尺寸和步长不一致，这样可以增加数据的丰富性
pool1 = tf.nn.max_pool(conv1,ksize = [1,3,3,1],strides = [1,2,2,1],
                      padding = 'SAME')
# 模仿了生物神经系统的“侧抑制”机制，对局部神经元的活动创建竞争环境，使得其中响应比较大的值变得相对更大
# 并抑制其中反馈较小的神经元，增强了模型的泛化能力
norm1 = tf.nn.lrn(pool1,4,bias = 1.0,alpha = 0.001/9.0,beta = 0.75)

# CONV2，64卷积核5x5

In [27]:
weight2 = variable_with_weight_loss(shape = [5,5,64,64],stddev = 5e-2,
                                   w1 = 0.0)
kernel2 = tf.nn.conv2d(norm1,weight2,[1,1,1,1],padding = 'SAME')
bias2 = tf.Variable(tf.constant(0.1,shape = [64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2,bias2))
norm2 = tf.nn.lrn(conv2,4,bias = 1.0,alpha = 0.001/9.0,beta = 0.75)
pool2 = tf.nn.max_pool(norm2,ksize = [1,3,3,1],strides = [1,2,2,1],
                      padding = 'SAME')

# FULL-CONNECTED, 384节点

In [28]:
# [batch_size,-1]中 -1 表示让系统自己计算，当然也可以手写2304
reshape = tf.reshape(pool2,[batch_size,-1])
# dim = 2304
dim = reshape.get_shape()[1].value
# 全链接层有384个节点
weight3 = variable_with_weight_loss(shape = [dim,384],stddev = 0.04,
                                    w1 = 0.004)
bias3 = tf.Variable(tf.constant(0.1,shape = [384]))
local3 = tf.nn.relu(tf.matmul(reshape,weight3) + bias3)

# FULL-CONNECTED, 192节点

In [29]:
weight4 = variable_with_weight_loss(shape = [384,192],stddev = 0.04,
                                    w1 = 0.004)
bias4 = tf.Variable(tf.constant(0.1,shape = [192]))
local4 = tf.nn.relu(tf.matmul(local3,weight4) + bias4)

# SOFTMAX 节点，10 类

In [36]:
weight5 = variable_with_weight_loss(shape = [192,10],stddev = 1/192.0,
                                    w1 = 0.0)
bias5 = tf.Variable(tf.constant(0.0,shape = [10]))
logits = tf.add(tf.matmul(local4,weight5),bias5)
print(logits.shape)

(128, 10)


In [44]:
# 计算正则化的loss
def loss(logits,labels):
    labels = tf.cast(labels,tf.int64)
    # 把 softmax 的计算和 cross entropy loss 的计算结合到一起
    # 注意！！！这个函数的返回值并不是一个数，而是一个向量，
    #如果要求交叉熵，我们要再做一步tf.reduce_sum操作,就是对向量里面所有元素求和，最后才得到，
    #如果求loss，则要做一步tf.reduce_mean操作，对向量求均值！
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = logits,labels = labels,name = 'cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,
                                       name = 'cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)
    # 对collection中所有元素求和，计算正则化loss
    return tf.add_n(tf.get_collection('losses'),name = 'total_loss')

In [45]:
loss = loss(logits,label_holder)

In [46]:
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

# top_k 的准确率

In [48]:
top_k_op = tf.nn.in_top_k(logits,label_holder,1)

In [49]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# 启动线程队列

In [50]:
tf.train.start_queue_runners()

[<Thread(Thread-4, started daemon 139743954204416)>,
 <Thread(Thread-5, started daemon 139743807403776)>,
 <Thread(Thread-6, started daemon 139743799011072)>,
 <Thread(Thread-7, started daemon 139743790618368)>,
 <Thread(Thread-8, started daemon 139743782225664)>,
 <Thread(Thread-9, started daemon 139743773832960)>,
 <Thread(Thread-10, started daemon 139743765440256)>,
 <Thread(Thread-11, started daemon 139743757047552)>,
 <Thread(Thread-12, started daemon 139743270532864)>,
 <Thread(Thread-13, started daemon 139743262140160)>,
 <Thread(Thread-14, started daemon 139743253747456)>,
 <Thread(Thread-15, started daemon 139743245354752)>,
 <Thread(Thread-16, started daemon 139743236962048)>,
 <Thread(Thread-17, started daemon 139743228569344)>,
 <Thread(Thread-18, started daemon 139743220176640)>,
 <Thread(Thread-19, started daemon 139743136315136)>,
 <Thread(Thread-20, started daemon 139743127922432)>,
 <Thread(Thread-21, started daemon 139743119529728)>,
 <Thread(Thread-22, started daemon

In [51]:
for step in range(max_steps):
    start_time = time.time()
    image_batch,label_batch = sess.run([images_train,labels_train])
    _,loss_value = sess.run([train_op,loss],
                           feed_dict = {image_holder:image_batch,label_holder:label_batch})
    duration = time.time() - start_time
    if step % 10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)
        
        format_str = ('step = %d,loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step,loss_value,examples_per_sec,sec_per_batch))

ResourceExhaustedError: OOM when allocating tensor with shape[128,24,24,64]
	 [[Node: Conv2D_2 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_Placeholder_2_0_0/_621, Variable_14/read)]]
	 [[Node: total_loss_1/_623 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_977_total_loss_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv2D_2', defined at:
  File "/home/lile/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/lile/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/lile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home

In [56]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch,label_batch = sess.run([images_test,labels_test])
    predictions = sess.run([top_k_op],feed_dict = {image_holder:image_batch,
                                                  label_holder:label_batch})
    true_count += np.sum(predictions)
    step += 1

In [ ]:
precision = true_count / total_sample_count
print('precision @ 1 = %.3f' % precision)